In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.optim
import torch.functional as F
from torch.utils.data import Dataset, DataLoader

import torchvision

import torchvision.transforms as transforms
import torch.optim as optim

import torch.nn as nn
from torch.autograd import Variable

from include.data import ObjectSegmentationDataset
from include.hybrid_net import SegmentationModel

from include.Utility_Functions import Validate_IOU,Validate_CrossEntropy

In [ ]:
# Loaders for Training and Validation

# It is important to normalise the Input images acording to
# https://pytorch.org/docs/stable/torchvision/models.html

transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean=[0.458, 0.438, 0.404],std=[0.266, 0.263, 0.278])])

train_dataset = ObjectSegmentationDataset(
    src_image_dir="/home/thanos/Desktop/Dataset/TRAIN/Data/",
    seg_image_dir="/home/thanos/Desktop/Dataset/TRAIN/Ground_Truth/",
    augment=True,
    rescale=(255,255),
    num_classes=21,
    transform=transform)

validation_dataset = ObjectSegmentationDataset(
     src_image_dir="/home/thanos/Desktop/Dataset/VAL/Data/",
     seg_image_dir="/home/thanos/Desktop/Dataset/VAL/Ground_Truth/",
     augment=False,
     rescale=False,
     num_classes=21,
     transform=transform)

bs=10
Train_Loader = torch.utils.data.DataLoader(train_dataset, batch_size=bs, shuffle=True, num_workers=4)
VAL_Loader = torch.utils.data.DataLoader(validation_dataset, batch_size=15, shuffle=False, num_workers=4)


In [ ]:
net = SegmentationModel(num_classes=20)
filename="/home/thanos/Desktop/Dataset/MODELS/IOU_Best_params.pth"
checkpoint = torch.load(filename)
net.load_state_dict(checkpoint['state_dict'])
net = net.cuda()

In [ ]:
best_IOU=0.0
loss_fn = nn.CrossEntropyLoss(ignore_index=20)
learning_rate=0.001
optimizer = optim.SGD([
                {'params': net.vgg.parameters(), 'lr': 1e-3},
                {'params': net.atr.parameters(), 'lr': 1e-2}
            ], momentum=0.9, weight_decay=0.0005)

lambda1 = lambda epoch: 0.001*(+1.0 - (epoch / 10000.0) )**0.9 
lambda2 = lambda epoch: 0.01*(+1.0 - (epoch / 10000.0) )**0.9 

Scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=[lambda1, lambda2], last_epoch=-1)
net.train()
losses=[]
IOU_Score=[]

for epoch in range(10000):
    
    epoch_loss=0.0
    
    for data in Train_Loader:
        src_img, _, seg_img, _ = data

        Input = Variable(src_img).float().cuda()
        Target = Variable(seg_img.long(), requires_grad=False).cuda()
        optimizer.zero_grad()
        Output = net(Input)
        

        loss = loss_fn(Output, Target)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss+=(loss.item())/10582.0

            
    # Checkpoint Network and Optimiser after each 25 epochs
    if (epoch % 25==0) :
        best_IOU,iou=Validate_IOU(net,optimizer,epoch,losses,IOU_Score,bs,learning_rate,VAL_Loader,best_IOU)
    
    losses.append(epoch_loss)
    IOU_Score.append(iou)
    Scheduler.step()
    print("Epoch = {} , Epoch_Loss = {} , Best_IOU = {} ".format(epoch,epoch_loss,best_IOU))

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,3,1)
plt.plot(losses)
plt.title("Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Cross Entropy")
plt.subplot(1,3,2)
plt.plot(IOU_Score)
plt.title("IOU in Validation Dataset")
plt.xlabel("Epoch")
plt.ylabel("IOU")